In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import class_weight
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

Carregando dados

In [ ]:
train_data = pd.read_csv('ReutersGrain-train.csv', escapechar='\\', quotechar="'")
test_data = pd.read_csv('ReutersGrain-test.csv', escapechar='\\', quotechar="'")

Pré-processamento

In [ ]:
def preprocess_text(text):
    # Remover pontuações
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenização e lematização
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in text.split()]
    return ' '.join(tokens)

In [ ]:
train_data['Text'] = train_data['Text'].str.replace('\n', ' ').str.lower().apply(preprocess_text)
test_data['Text'] = test_data['Text'].str.replace('\n', ' ').str.lower().apply(preprocess_text)

X_train, y_train = train_data['Text'], train_data['class-att']
X_test, y_test = test_data['Text'], test_data['class-att']

Vetorização do texto

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
y_train.value_counts()
y_test.value_counts()

Ajustando os pesos

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', classes=[0, 1], y=y_train)
clf = MultinomialNB(class_prior=class_weights)

Modelo Naive Bayes

In [ ]:
clf.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = clf.predict(X_test_tfidf)

Avaliação do Modelo

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification:\n", classification_report(y_test, y_pred))